In [2]:
import pandas as pd
import numpy as np

In [3]:
# warning 무시
import warnings
warnings.filterwarnings(action='ignore')

In [4]:
reserve_tb=pd.read_csv('./data/reserve.csv',encoding='UTF-8')
reserve_tb.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100


--------
### 3.1 데이터와 종류의 개수 산출
#### hotel_id를 키 값으로 하여 고객수와 예약수를 집계해보자 (Not Awesome)
* groupby(), size(), unique(), merge()

In [5]:
# 현재 행 번호로 고친다.
rsv_cnt_tb=reserve_tb.groupby('hotel_id').size().reset_index()
rsv_cnt_tb.head()

,hotel_id,0
0,h_1,10
1,h_10,3
2,h_100,20
3,h_101,17
4,h_102,13


In [6]:
# 열 이름 변경
rsv_cnt_tb.columns=['hotel_id','rsv_cnt']
rsv_cnt_tb.head()

,hotel_id,rsv_cnt
0,h_1,10
1,h_10,3
2,h_100,20
3,h_101,17
4,h_102,13


In [7]:
# hotel_id를 집약 단위로 설정하고, customer_id값을 nunique함수를 이용하여 고객 수 카운팅
cus_cnt_tb=reserve_tb.groupby('hotel_id')['customer_id'].nunique().reset_index()
cus_cnt_tb.head()

,hotel_id,customer_id
0,h_1,10
1,h_10,3
2,h_100,19
3,h_101,17
4,h_102,13


In [8]:
# 열 이름 변경
cus_cnt_tb.columns=['hotel_id','cus_cnt']
cus_cnt_tb.head()

,hotel_id,cus_cnt
0,h_1,10
1,h_10,3
2,h_100,19
3,h_101,17
4,h_102,13


In [9]:
# merge함수를 이용해 hotel_id를 결합 키로 결합
pd.merge(rsv_cnt_tb, cus_cnt_tb, on='hotel_id').head()

,hotel_id,rsv_cnt,cus_cnt
0,h_1,10,10
1,h_10,3,3
2,h_100,20,19
3,h_101,17,17
4,h_102,13,13


#### hotel_id를 키 값으로 하여 고객수와 예약수를 집계해보자 (Awesome)
* groupby(), agg(), reset_index()

In [21]:
# agg함수를 사용해서 통계량 구하기
reserve_tb.groupby('hotel_id').agg(['size','mean','std','min','max']).head()

people_num                             total_price                 \
               size      mean       std min max        size           mean   
hotel_id                                                                     
h_1              10  2.600000  1.264911   1   4          10  112230.000000   
h_10              3  2.666667  1.527525   1   4           3   42933.333333   
h_100            20  2.700000  1.080935   1   4          20   27600.000000   
h_101            17  2.411765  1.227743   1   4          17   75764.705882   
h_102            13  2.769231  1.012739   1   4          13   32769.230769   

                                       
                   std    min     max  
hotel_id                               
h_1       56449.526127  26100  208800  
h_10      28736.968061  11200   67200  
h_100     17883.835689   4800   57600  
h_101     49014.703676  14000  168000  
h_102     18912.755159  12000   72000

In [11]:
# rserve_id를 대상으로 count 함수를 적용
# customer_id를 대상으로 ununique함수를 적용
result=reserve_tb.groupby('hotel_id').agg({'reserve_id':'count', 'customer_id':'nunique'})

# reset_index함수로 열 번호를 다시 설정한다.
# inplace=True이므로 컬럼명 다시 갱신
result.reset_index(inplace=True)
result.columnns=['hotel_id','rsv_cnt','cus_cnt']
result.head()

,hotel_id,reserve_id,customer_id
0,h_1,10,10
1,h_10,3,3
2,h_100,20,19
3,h_101,17,17
4,h_102,13,13


----------------
### 3.2 합곗값 계산

In [12]:
# 호텔별로 숙박예약 인원 수에 따른 매출 금액 합 
result=reserve_tb.groupby(['hotel_id','people_num'])['total_price'].sum().reset_index()
result.head()

,hotel_id,people_num,total_price
0,h_1,1,156600
1,h_1,2,156600
2,h_1,3,391500
3,h_1,4,417600
4,h_10,1,11200


변경할 열이 적다면 rename을 사용하는 것이 좋음.

In [13]:
result.rename(columns={'total_price':'price_sum'}).head()

,hotel_id,people_num,price_sum
0,h_1,1,156600
1,h_1,2,156600
2,h_1,3,391500
3,h_1,4,417600
4,h_10,1,11200


아래와 같이 변경해야 할 열 이름이 많을 때는 직접 DataFrame의 column을 변경하는 것이좋음.

In [14]:
result.columns=['hotel_id_re','people_num_re','total_price_re']

### 3.3 최댓값, 최솟값, 대푯값 산출

호텔별 예약 금액의 최댓값, 최솟값, 평균값, 중앙값, 20백분위값 산출

In [15]:
result = reserve_tb.groupby('hotel_id').agg({'total_price':['max','min','mean','median', lambda x:np.percentile(x,q=20)]}).reset_index()
result.columns = ['hotel_id','price_max','price_min','price_mean','price_median','price_20per']
result.head()

,hotel_id,price_max,price_min,price_mean,price_median,price_20per
0,h_1,208800,26100,112230.000000,104400,73080
1,h_10,67200,11200,42933.333333,50400,26880
2,h_100,57600,4800,27600.000000,28800,9600
3,h_101,168000,14000,75764.705882,56000,30800
4,h_102,72000,12000,32769.230769,24000,18000


### 3.4 분포 계산
total_price를 var함수와 std함수에 적용하여 분산값과 표준편차값 계산.

In [16]:
result=reserve_tb.groupby('hotel_id').agg({'total_price':['var','std']}).reset_index()
result.columns=['hotel_id','price_var','price_std']
result.head()

,hotel_id,price_var,price_std
0,h_1,3.186549e+09,56449.526127
1,h_10,8.258133e+08,28736.968061
2,h_100,3.198316e+08,17883.835689
3,h_101,2.402441e+09,49014.703676
4,h_102,3.576923e+08,18912.755159


In [17]:
# 데이터가 한건이면 분산과 표준편차가 NA가 되며, fillna함수를 이용해 0으로 변환.
result.fillna(0,inplace=True)

In [18]:
# 특정 열의 NA값만 변환할 때
result.fillna(value={'price_var':0, 'price_std':0}, inplace=True)

### 3.5 최빈값 계산
예약 테이블의 예약 금액을 1000단위의 범주로 변환하여 최빈값 계산

In [19]:
# round함수로 반올림한 후 mode함수로 최빈값 계산
reserve_tb['total_price'].round(-3).mode()          # 셋째자리에서 반올림

0    10000
1    20000
2    40000
dtype: int64

### 3.6 순위 계산
customer_id별로 reserve_datetime순으로 log_no로서 순위를 부여

#### 3.6.1 시간 데이터에 번호 부여

In [25]:
# 변경 전 data type
reserve_tb['reserve_datetime'].dtype

dtype('O')

In [30]:
# rank함수는 문자형을 처리할 수 없으므로, 자료형을 문자열에서 timestamp형으로 변환한다.
reserve_tb['reserve_datetime']=pd.to_datetime(reserve_tb['reserve_datetime'], format='%Y-%m-%d %H:%M:%S')

In [31]:
# 변경 후 data type
reserve_tb['reserve_datetime'].dtype

dtype('<M8[ns]')

In [41]:
reserve_tb['log_no']=reserve_tb.groupby('customer_id')['reserve_datetime'].rank(ascending=True, method='first')

In [45]:
reserve_tb[['customer_id','reserve_datetime','log_no']].head(20)

,customer_id,reserve_datetime,log_no
0,c_1,2016-03-06 13:09:42,1.0
1,c_1,2016-07-16 23:39:55,2.0
2,c_1,2016-09-24 10:03:17,3.0
3,c_1,2017-03-08 03:20:10,4.0
4,c_1,2017-09-05 19:50:37,5.0
5,c_1,2017-11-27 18:47:05,6.0
6,c_1,2017-12-29 10:38:36,7.0
7,c_1,2018-05-26 08:42:51,8.0
8,c_2,2016-03-05 13:31:06,1.0
9,c_2,2016-06-25 09:12:22,2.0


#### 3.6.2 랭킹
호텔별 예약 건수에 따른 순위를 매겨보자.

In [53]:
# 예약 횟수를 계산한다
rsv_cnt_tb = reserve_tb.groupby('hotel_id').size().reset_index()
rsv_cnt_tb.columns=['hotel_id','rsv_cnt']

In [59]:
# 예약 횟수를 기준으로 순위 계산
## ascending=False로 하여 숫자가 큰거부터 1,2,3,,,, 
### method=min 으로 지정하여, 값이 같은 경우엔 최소 순위를 지정
rsv_cnt_tb['rsv_cnt_rank']=rsv_cnt_tb['rsv_cnt'].rank(ascending=False, method='min')

In [67]:
rsv_cnt_tb

,hotel_id,rsv_cnt,rsv_cnt_rank
0,h_1,10,235.0
1,h_10,3,300.0
2,h_100,20,12.0
3,h_101,17,43.0
4,h_102,13,139.0


In [68]:
# 불필요한 rsv_cnt 컬럼은 제거
rsv_cnt_tb.drop('rsv_cnt', axis=1, inplace=True)

In [70]:
rsv_cnt_tb.head()

,hotel_id,rsv_cnt_rank
0,h_1,235.0
1,h_10,300.0
2,h_100,12.0
3,h_101,43.0
4,h_102,139.0
